In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math
from scipy import optimize

import matplotlib as mpl
from matplotlib import cm
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'

import uposham.turning_point_coord_difference as tpcd
import gezeltermiller_hamiltonian as gm

import os
path_to_data = 'data/'
path_to_saveplot = 'plots/'

In [2]:
#%% Setting up parameters and global variables
save_final_plot = True
show_final_plot = False
show_itrsteps_plots = False # show iteration of the UPOs in plots
N = 4         # dimension of phase space

masses = [16, 1]
params_pe = [
    1.0074e-2,  # k
    1.9769e0,   # d
    -2.3597e-3, # a2
    1.0408e-3,  # a4
    -7.5496e-5, # a6
    7.7569e-3,  # c
    -2.45182e-4 # d0
            ]

parameters = masses + params_pe

eqNum = 3
eqPt = tpcd.get_eq_pts(eqNum, gm.init_guess_eqpt_gm, \
                        gm.grad_pot_gm, parameters)
e_saddle = gm.pot_energy_gm(eqPt[0], eqPt[1], parameters)
print("Eq. point:", eqPt)
print("Eq. energy:", e_saddle)

Eq. point: [-0.54669686  0.00217407]
Eq. energy: 0.0006697342077263631


In [3]:
#E_vals = [1.1, 2.00, 3.00, 5.00]
#linecolor = ['b','r','g','m','c']
E_vals = [0.01, 0.02]
linecolor = ['b','r']

n = 8 # number of intervals we want to divide.
n_turn = 1 # nth turning point we want to choose.
    
for i in range(len(E_vals)):
    
    e = E_vals[i] # total energy.
    deltaE = e - e_saddle
    
    #Trial initial Condition s.t. one initial condition is on the LHS of the UPO 
    #and the other one is on the RHS of the UPO
    
    trial_init_1 = eqPt[0] + 0.01
    trial_init_2 = eqPt[0] - 0.01
    
    
    f1 = lambda y: gm.get_coord_gm(trial_init_1,y,e,parameters)
    y0_2 = optimize.newton(f1,-0.15)
    state0_2 = [eqPt[0], y0_2,trial_init_1,0.0]

    f2 = lambda y: gm.get_coord_gm(trial_init_1,y,e,parameters)
    y0_3 = optimize.newton(f2,-0.15)
    state0_3 = [eqPt[0], y0_3,trial_init_2,0.0]
    
    with open("x0_tpcd_deltaE%s_gm.dat" %(deltaE),'a+') as po_fam_file:
        [x0po_1, T_1,energyPO_1] = tpcd.turningPoint_configdiff(
            state0_2, state0_3, gm.get_coord_gm, \
            gm.pot_energy_gm, gm.variational_eqns_gm, \
            gm.configdiff_gm, \
            gm.ham2dof_gm, \
            gm.half_period_gm, \
            gm.guess_coords_gm, \
            gm.plot_iter_orbit_gm, \
            parameters, e, n, n_turn, show_itrsteps_plots, po_fam_file) 



h is  0.0
Initial guess1 [-0.5466968637011862, -1.3590333210519838, -0.5366968637011862, 0.0], initial guess2 [-1.4454826860896342, -1.3590333210519838, 0, 0],             y_diff1 is -3.8349663106091882, y_diff2 is -2.751802417883252
h is  0.0
Initial guess1 [-0.5466968637011862, -1.3590333210519838, -0.5366968637011862, 0.0], initial guess2 [-1.4454826860896342, -1.3590333210519838, 0, 0],             y_diff1 is -3.8349663106091882, y_diff2 is -2.751802417883252
h is  0.0
Initial guess1 [-0.5466968637011862, -1.3590333210519838, -0.5366968637011862, 0.0], initial guess2 [-1.4454826860896342, -1.3590333210519838, 0, 0],             y_diff1 is -3.8349663106091882, y_diff2 is -2.751802417883252
h is  0.0
Initial guess1 [-0.5466968637011862, -1.3590333210519838, -0.5366968637011862, 0.0], initial guess2 [-1.4454826860896342, -1.3590333210519838, 0, 0],             y_diff1 is -3.8349663106091882, y_diff2 is -2.751802417883252
h is  0.0
Initial guess1 [-0.5466968637011862, -1.35903332105198

Initial guess1 [0. 0. 0. 0.], initial guess2 [-2.7886814186357043, 0.0, 0, 0],             y_diff1 is 0.0, y_diff2 is -2.4816035804862215
h is  0.0
Initial guess1 [0. 0. 0. 0.], initial guess2 [-2.7886814186357043, 0.0, 0, 0],             y_diff1 is 0.0, y_diff2 is -2.4816035804862215
h is  0.0
Initial guess1 [0. 0. 0. 0.], initial guess2 [-2.7886814186357043, 0.0, 0, 0],             y_diff1 is 0.0, y_diff2 is -2.4816035804862215


In [4]:

#%% Load periodic orbit data from ascii files
    
x0po = np.zeros((4,len(E_vals))) #each column is a different initial condition

for i in range(len(E_vals)):
    
    e = E_vals[i]
    deltaE = e - e_saddle

    with open("x0_tpcd_deltaE%gm.dat" %(deltaE),'a+') as po_fam_file:
        print('Loading the periodic orbit family from data file',po_fam_file.name,'\n') 
        x0podata = np.loadtxt(po_fam_file.name)
        x0po[:,i] = x0podata[-1,0:4] 



Loading the periodic orbit family from data file x0_tpcd_deltaE0.00933027m.dat 



/Users/telegramsam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: loadtxt: Empty input file: "x0_tpcd_deltaE0.00933027m.dat"
  if sys.path[0] == '':


IndexError: too many indices for array

In [ ]:

#%% Plotting the Family
    
TSPAN = [0,400]
plt.close('all')
axis_fs = 15
RelTol = 3.e-10
AbsTol = 1.e-10

f = lambda t,x : gm.ham2dof_gm(t,x,parameters) 

ax = plt.gca(projection='3d')

for i in range(len(E_vals)):
    
    e = E_vals[i]
    deltaE = e - e_saddle
    
    soln = solve_ivp(f, TSPAN, x0po[:,i], method='RK45', dense_output=True, \
                     events = lambda t,x : gm.half_period_gm(t,x,parameters), \
                     rtol=RelTol, atol=AbsTol)
    
    te = soln.t_events[0]
    tt = [0,te[2]]
    t,x,phi_t1,PHI = tpcd.state_transit_matrix(tt, x0po[:,i], parameters, \
                                        gm.variational_eqns_gm)
    
    ax.plot(x[:,0],x[:,1],x[:,2],'-',color=linecolor[i], \
            label='$\Delta E$ = %.2f'%(deltaE))
    ax.scatter(x[0,0],x[0,1],x[0,2],s=10,marker='*')
    ax.plot(x[:,0], x[:,1], zs=0, zdir='z')
    # ax.plot(x[:,0], x[:,1], zs=0, zdir='z') # 2D projection of the UPO

    
resX = 100
xVec = np.linspace(-1,1,resX)
yVec = np.linspace(-2,2,resX)
xMat, yMat = np.meshgrid(xVec, yVec)
cset1 = ax.contour(xMat, yMat, \
                    tpcd.get_pot_surf_proj(
                       xVec, yVec, gm.pot_energy_gm, \
                       parameters), \
                    [0.01,0.1,1,2,4], zdir='z', offset=0, \
                    linewidths = 1.0, cmap=cm.viridis, \
                    alpha = 0.8)

ax.scatter(eqPt[0], eqPt[1], s = 50, c = 'r', marker = 'X')
ax.set_xlabel('$x$', fontsize=axis_fs)
ax.set_ylabel('$y$', fontsize=axis_fs)
ax.set_zlabel('$p_x$', fontsize=axis_fs)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-4, 4)
legend = ax.legend(loc='upper left')

plt.grid()

if show_final_plot:
    plt.show()

if save_final_plot:  
    plt.savefig( path_to_saveplot + 'tpcd_gm_upos.pdf', \
                format='pdf', bbox_inches='tight')
